In [1]:
import sys
sys.path.append("..")

from pathlib import Path

from src.intent.data_manager import IntentDataManager
from src.intent.models import IntentClassifier

In [3]:
data_manager = IntentDataManager(
    cache_dir=Path("../cache/intent"),
    max_len=128,
    batch_size=32,
    num_workers=8,
    data_dir=Path("../data/intent"),
    test_file=Path("../data/intent/test.json")
)

2022-03-10 02:21:13 | INFO | Vocab loaded from /home/jacky/110-2_ADL/homeworks/hw01/cache/intent/vocab.pkl
2022-03-10 02:21:13 | INFO | Intent-2-Index loaded from /home/jacky/110-2_ADL/homeworks/hw01/cache/intent/intent2idx.json
2022-03-10 02:21:13 | INFO | Embeddings loaded from /home/jacky/110-2_ADL/homeworks/hw01/cache/intent/embeddings.pt


In [4]:
valid_dataloader = data_manager.get_valid_dataloader()
valid_dataloader

In [5]:
x, length, y = next(iter(valid_dataloader))
x.shape, length.shape, y.shape

(torch.Size([32, 24]), torch.Size([32]), torch.Size([32]))

In [6]:
model = IntentClassifier(
    embeddings=data_manager.embeddings,
    hidden_size=128,
    num_layers=2,
    dropout=.2,
    bidirectional=True,
    num_class=data_manager.num_class,
    lr=1e-3,
    weight_decay=1e-5
)
model

IntentClassifier(
  (embedding): Embedding(5485, 300, padding_idx=0)
  (rnn): GRU(300, 128, num_layers=2, dropout=0.2, bidirectional=True)
  (fc): Sequential(
    (0): Dropout(p=0.2, inplace=False)
    (1): Linear(in_features=256, out_features=256, bias=True)
    (2): PReLU(num_parameters=1)
    (3): Dropout(p=0.2, inplace=False)
    (4): Linear(in_features=256, out_features=150, bias=True)
  )
  (loss): CrossEntropyLoss()
)

In [8]:
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

embeddings = model.embedding(x)
packed_features = pack_padded_sequence(
    input=embeddings,
    lengths=length.cpu(),
    batch_first=True,
    enforce_sorted=False
)
packed_output_features, _ = model.rnn(packed_features)
output_features, _ = pad_packed_sequence(
    sequence=packed_output_features,
    batch_first=True
)

In [29]:
import torch
# Non-bidirectional
torch.vstack([
    feature[sen_len-1, :]
    for feature, sen_len in zip(output_features, length)
]).shape, output_features[:, -1, :].shape

(torch.Size([32, 256]), torch.Size([32, 256]))

In [37]:
# Bidirectional
forward_features, backward_features = torch.chunk(output_features, 2, dim=2)
last_forward_features = torch.vstack([
    feature[sen_len-1, :]
    for feature, sen_len in zip(forward_features, length)
])
torch.cat((last_forward_features, backward_features[:, 0, :]), dim=1).shape

torch.Size([32, 256])